In [68]:
import os
import cv2
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from skimage import exposure
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D

In [69]:
def extract_hog_features(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Extract HOG features
    features, hog_image = hog(gray_image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    
    # Perform global contrast normalization
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    
    return features

In [70]:
local = 'cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [79]:
# Set random seed for reproducibility
np.random.seed(14)

# Create data generators for both training and testing data
def preprocess_images(directory):
    images = []
    labels = []
    
    for category in os.listdir(directory):
        category_dir = os.path.join(directory, category)
        
        for filename in os.listdir(category_dir):
            image_path = os.path.join(category_dir, filename)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (150, 150))
            
            # Extract HOG features
            hog_features = extract_hog_features(image)
            
            images.append(hog_features)
            labels.append(1 if category == 'dogs' else 0)
    
    return np.array(images), np.array(labels)

train_images, train_labels = preprocess_images(train_dir)
validation_images, validation_labels = preprocess_images(validation_dir)

# Create ANN model
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(train_images.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train ANN on training data
model.fit(train_images, train_labels, epochs=14, batch_size=32)

Epoch 1/14
63/63 [==============================] - 6s 70ms/step - loss: 0.6798 - accuracy: 0.5745
Epoch 2/14
63/63 [==============================] - 4s 65ms/step - loss: 0.6107 - accuracy: 0.6715
Epoch 3/14
63/63 [==============================] - 4s 67ms/step - loss: 0.5757 - accuracy: 0.7030
Epoch 4/14
63/63 [==============================] - 5s 76ms/step - loss: 0.5526 - accuracy: 0.7185
Epoch 5/14
63/63 [==============================] - 4s 64ms/step - loss: 0.5265 - accuracy: 0.7455
Epoch 6/14
63/63 [==============================] - 5s 77ms/step - loss: 0.4994 - accuracy: 0.7745
Epoch 7/14
63/63 [==============================] - 4s 66ms/step - loss: 0.4881 - accuracy: 0.7740
Epoch 8/14
63/63 [==============================] - 4s 60ms/step - loss: 0.4826 - accuracy: 0.7755
Epoch 9/14
63/63 [==============================] - 4s 58ms/step - loss: 0.4679 - accuracy: 0.7895
Epoch 10/14
63/63 [==============================] - 4s 58ms/step - loss: 0.4442 - accuracy: 0.8090
Epoch 11/

In [80]:
# Evaluate the performance of the ANN on the validation data
validation_loss, validation_accuracy = model.evaluate(validation_images, validation_labels)
print(f"Akurasi Validasi: {validation_accuracy * 100:.2f}%")

32/32 [==============================] - 0s 4ms/step - loss: 0.5570 - accuracy: 0.7200
Akurasi Validasi: 72.00%
